In [1]:
!pip install -q streamlit

In [2]:
!pip install -q streamlit-ace

In [3]:
!streamlit run app.py &>/dev/null&

In [4]:
!pip install -q pyngrok

In [5]:
!pip install cvlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
!pip install cv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
!pip install captcha

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
!pip install opencv-python tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
from pyngrok import ngrok

In [47]:
%%writefile app.py
import streamlit as st
from streamlit import session_state as _state
from captcha.image import ImageCaptcha
import string
import random
import cv2
import cvlib as cv
from cvlib.object_detection import draw_bbox
from PIL import Image
import numpy as np
from io import StringIO


GENDER_OPTIONS = {
    'Empresa' : 0,
    'Feminino' : 1,
    'Masculino' : 2,
    'Não Identificar' : 3,
}

GENDER_OPTIONS_COMBOBOX = list(GENDER_OPTIONS.keys())

def set_page_to_form() :
  return st.session_state.update({"page": "form"})

def set_page_to_image() :
  return st.session_state.update({"page": "image"})

def code_generator(size=6, chars=string.ascii_uppercase + string.digits):
    return "".join(random.choice(chars) for _ in range(size))

def handle_picture(picture):
    return detect_on_image(picture)

def detect_on_image(image):
  bbox, label, conf = cv.detect_common_objects(image, confidence=0.2)
  return draw_bbox(image, bbox, label, conf), label

IMAGE = ImageCaptcha()
CAPTCHA_CODE = code_generator()

################################################3

_PERSIST_STATE_KEY = f"{__name__}_PERSIST"

def persist(key: str) -> str:
    if _PERSIST_STATE_KEY not in _state:
        _state[_PERSIST_STATE_KEY] = set()

    _state[_PERSIST_STATE_KEY].add(key)

    return key


def load_widget_state():
    if _PERSIST_STATE_KEY in _state:
        _state.update({
            key: value
            for key, value in _state.items()
            if key in _state[_PERSIST_STATE_KEY]
        })

####################################

def main() :
  if "page" not in st.session_state:
    st.session_state.update({
      "page": "form",
      "name" : "",
      "sobrenome" : "",
      "gender" : "Empresa",
      "age" : "",
      "document" : ""
    })

  st.set_page_config(page_title="Santander", page_icon="🏷️", layout="centered")

  PAGES[st.session_state.page]()

def form() :

  st.title("Cadastro")

  form = st.form(key="form")

  with st.form("Cadastro") :
      name = st.text_input("Nome Completo:", key=persist("name"))
      document = st.text_input("Documento:", key=persist("document"))

      colUserInfo = st.columns((2,1))
      gender = colUserInfo[0].selectbox("Gênero:", GENDER_OPTIONS_COMBOBOX, key=persist("gender"))
      age = colUserInfo[1].text_input("Idade:", key=persist("age"))

      captcha_image = IMAGE.generate('AI4UQA')
      st.image(captcha_image, use_column_width=True)
      captcha_answer = st.text_input("Captcha")
      button = st.form_submit_button(label="Confirmar")

      if button:
        if captcha_answer == 'AI4UQA':
          set_page_to_image()
        else:
          st.warning("Erro no Captcha, Tente Novamente!")

def image() :

  st.title("Verificação de Imagem")

  form = st.form(key="form")

  with st.form("Verificação de Imagem"):
    image_file = st.file_uploader("Envie uma Imagem Segurando um Garfo!", type=["png","jpg","jpeg"])

    colb1, colb2 , colb3 = st.columns(3)

    with colb1:
        backToHome = st.form_submit_button(label="Retornar")
    with colb2:
        pass
    with colb3:
        submitted = st.form_submit_button(label="Confirmar Envio")

    if backToHome:
      set_page_to_form()
    if submitted:
      nparr = np.fromstring(image_file.getvalue(), np.uint8)
      img_np = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

      img, label = handle_picture(img_np)
      st.image(img, use_column_width=True)

      if 'person' in label and 'fork' in label:
        st.success('Cadastro Realizado com Sucesso!')
      else:
        st.error('Erro no Imagem, Por Favor Envie Outra Imagem Conforme Pedido!')
  
PAGES = {
    "form": form,
    "image": image,
}

if __name__ == "__main__":
  load_widget_state()
  main()

Overwriting app.py


In [48]:
!streamlit run app.py & npx localtunnel --port 8501

2022-08-28 22:15:48.353 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
npx: installed 22 in 2.524s
your url is: https://green-eggs-add-35-245-31-20.loca.lt

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.245.31.20:8501

/content/app.py:126: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  nparr = np.fromstring(image_file.getvalue(), np.uint8)
/content/app.py:126: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  nparr = np.fromstring(image_file.getvalue(), np.uint8)
/content/app.py:126: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  nparr = np.fromstring(image_file.getvalue(), np.uint8)
  Stopping...
^C
